In [56]:
import numpy as np
import matplotlib.pyplot as plt

# 定义常量和参数
X1 = 0
X2 = 2
Y1 = X2 + 0.01
Y2 = 30 + 0.01
n = int((X2 - X1) * 1000 + 1)
m = int((Y2 - Y1) * 100 + 1)

hx = (X2 - X1) / (n - 1)
hy = (Y2 - Y1) / (m - 1)
x = np.linspace(X1, X2, n)
y = np.linspace(Y1, Y2, m)


In [57]:
# 定义函数 f
def f(x, M, G, F):
    return F ** 2 * ((M ** 4 + M ** 2 * G ** 2) / (24 * np.pi * ((x - M ** 2) ** 2 + M ** 2 * G ** 2)))


# 计算 f1, f2, f3, f4
m1 = 0.78
gamma1 = 0.15
m2 = 1.46
gamma2 = 0.4
m3 = 1.72
gamma3 = 0.25
m4 = 1.90
gamma4 = 0.1

f1 = f(x, m1, gamma1, 1)
f2 = f(x, m2, gamma2, 1)
f3 = f(x, m3, gamma3, 1)
f4 = f(x, m4, gamma4, 1)

F = f1 + f2 + f3 + f4

In [ ]:
# 计算 G1 和 G2
def G1_function(x, F1, y):
    return np.trapz(F1.dot(1 / (y - x)), x)


def G2_function(x):
    return x - x


G1 = np.array([G1_function(x, f1, y_i) for y_i in y])
G2 = np.array([G2_function(x) for _ in range(m)])

In [ ]:
np.trapz(1 / (8 - x) * F1, x)


In [55]:
F1.dot(1 / (8 - x))


16.980472229525063

In [ ]:

B1 = 1
B2 = 2
XD = 1.01
b1 = XD * B1
b2 = XD * B2


In [ ]:
# 计算 f_exact 和 f_delta
f_exact = B1 * f1 + B2 * f2
f_delta = b1 * f1 + b2 * f2

# 计算 g_exact 和 g_delta
g_exact = B1 * G1 + B2 * G2
g_delta = b1 * G1 + b2 * G2

# 计算 V_exact
MM = f_delta[0]
NN = f_delta[-1]
V_exact = (MM * (x - X2)) / (X1 - X2) + (NN * (x - X1)) / (X2 - X1)


In [ ]:
# 计算 G3
G3 = np.zeros(m)
for i in range(m):
    G3[i] = np.trapz(1 / (y[i] - x) * V_exact, x)

# 计算 U_exact 和 G_exact
U_exact = f_exact - V_exact
G_exact = g_exact - G3
G_delta = g_delta - G3


In [ ]:
# 计算 Aphi
Aphi = np.zeros((m, n))
for j in range(1, n - 1):
    Aphi[:, j] = 1 / hx * ((y - x[j - 1]) * np.log(y - x[j - 1]) + (x[j - 1] - x[j]) +
                           (y - x[j + 1]) * np.log(y - x[j + 1]) - 2 * (y - x[j]) * np.log(y - x[j]) -
                           (x[j] - x[j + 1]))
Aphi[:, 0] = np.log(y - x[0]) + 1 / hx * ((y - x[1]) * np.log(y - x[1]) -
                                          (y - x[0]) * np.log(y - x[0]) - (x[0] - x[1]))
Aphi[:, -1] = -np.log(y - x[-1]) + 1 / hx * (-(y - x[-1]) * np.log(y - x[-1]) +
                                             (y - x[-2]) * np.log(y - x[-2]) + (x[-2] - x[-1]))

In [62]:
Aphi[:, -1].dot(Aphi[:, -1])

0.003922373181307892

In [ ]:
# 计算 A 矩阵和 ba 向量
A = np.zeros((n, n))
ba = np.zeros(n)
for i in range(n):
    for j in range(n):
        A[i, j] = np.trapz(Aphi[:, i] * Aphi[:, j], y)
    ba[i] = np.trapz(Aphi[:, i] * g_delta, y)

In [ ]:

# 计算 M 和 M1 矩阵
M = np.zeros((n, n))
M[0, 0] = 4
M[0, 1] = 2
for i in range(1, n - 1):
    M[i, i] = 8
    M[i, i - 1] = 2
    M[i, i + 1] = 2
M[n - 1, n - 1] = 4
M[n - 1, n - 2] = 2
M = hx / 12 * M

M1 = np.zeros((n, n))
M1[0, 0] = 1
M1[0, 1] = -1
for i in range(1, n - 1):
    M1[i, i] = 2
    M1[i, i - 1] = -1
    M1[i, i + 1] = -1
M1[n - 1, n - 1] = 1
M1[n - 1, n - 2] = -1
M1 = (1 / hx) * M1

H1 = M + M1

# 计算 alpha 取值范围
ite = 20
alpha = [10 ** (-k) for k in range(ite)]
ua = np.zeros((n, ite))
Ua = np.zeros((n, ite))
fa = np.zeros((n, ite))
res = np.zeros(ite)
err = np.zeros(ite)
L = np.zeros(ite)
LH = np.zeros(ite)

# 计算 L 曲线和 LH 曲线
for k in range(ite):
    ua[:, k] = np.linalg.solve(A + alpha[k] * H1, ba)
    Ua[:, k] = np.concatenate(([0], ua[:, k], [0]))
    fa[:, k] = Ua[:, k] + V_exact
    res[k] = np.sqrt(np.trapz((Aphi.dot(fa[:, k]) - g_delta) ** 2, y))
    err[k] = np.sqrt(np.trapz((fa[:, k] - f_exact) ** 2, x))
    L[k] = np.sqrt(np.trapz(fa[:, k] ** 2, x)) * res[k]
    LH[k] = np.sqrt(
        np.trapz(fa[:, k] ** 2 + np.gradient(fa[:, k]) ** 2, x)) * res[k]

# 画图
for k in range(ite):
    plt.subplot(4, 5, k + 1)
    plt.plot(x, f_exact, label='Exact')
    plt.plot(x, fa[:, k], label=f'Alpha = {alpha[k]}', linewidth=3)
    plt.legend()
    plt.title(f'Alpha = {alpha[k]}')

# 寻找最小值
kl = np.argmin(np.abs(L))
klh = np.argmin(np.abs(LH))

# 计算 V 和 Rho
V = np.zeros(20)
for i in range(20):
    VV = np.eye(len(y)) - Aphi.dot(np.linalg.solve(Aphi.T.dot(Aphi) +
                                                   alpha[i] * np.eye(len(x)), Aphi.T))
    V1 = np.linalg.norm(VV.dot(g_delta)) ** 2
    V2 = np.linalg.norm(VV) ** 2
    V[i] = V1 / V2

Vm, Vi = min(V)

Rho = np.zeros(20)
for i in range(20):
    Rho[i] = np.linalg.norm(
        alpha[i] * (Aphi.T.dot(Aphi) + alpha[i] * np.eye(len(x))).dot(fa[:, i]))

Rm, Ri = min(Rho)

result = {
    'L-curve+L^2': kl,
    'L-curve+H^1': klh,
    'GCV': Vi,
    '拟最优': Ri
}

print(result)